# 装饰器

## 概念

装饰是为函数和类指定管理或扩增代码的一种方式。装饰器本身采用可调用对象的形式，并处理其他可调用对象

- 函数装饰器： 在函数定义时进行名称重绑定，提供一个逻辑层来管理函数和方法，以及管理随后对它们的调用
- 类装饰器：在类定义的时候进行名称重绑定，提供一个逻辑层来管理类，以及管理随后调用它们所创建的实例

总而言之，装饰器提供了一种方法，在函数和类定义语句结束时插入自动运行的代码。

## 基础

### 函数装饰器

函数装饰器是一种关于函数的运行时声明，函数的定义需要遵守此声明。装饰器在定义函数或方法的def语句的前一行编写，并且它由@符号以及紧随其后的对于元函数的一个引用组成-这是管理另一个函数的函数

```python
@decorator
def F(args):
    ...
F(99)

# 等价于
def F(args):
    ...
F = decorator(F)
F(99)
```

### 类装饰器

类装饰器类似函数装饰器，只是它是管理类的一种方式，或是使用额外逻辑来完成实例构造调用的一种方式，这些额外的逻辑管理或扩展从类中创建的实例。

```python 
@decorator
class C:
    ...
x = C(99)
```

In [1]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)

        def __getattr__(self, name):
            return getattr(self.wrapped, name)
    return Wrapper

In [2]:
@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

In [3]:
x = C(6, 7)
print(x.attr)

spam


装饰器把类的名称重新绑定到另一个类，这个类在外层作用域中保持了最初的类，并且当调用的时候，这个类创建并嵌入了最初类的一个实例。当之后从该实例获取一个属性的时候，包装器的`__getattr__`拦截了它，并且将其委托给最初的类的嵌入实例。

### 装饰器嵌套
```python
@A
@B
def f():...

def f(...):...
f = A(B(f))
```

### 装饰器参数

函数装饰器和类装饰器都能接受参数，但是实际上这些参数传递给了返回装饰器的一个可调用对象，而装饰器反过来又返回一个可调用对象。
```python 
@decorator(A, B)
def F(arg):...
F(99)

####################
def F(args):...
F = decorator(A, B)(F)

F(99)
```
装饰器参数往往意味着可调用对象的三个层级：
接受装饰器参数的一个可调用对象，它返回一个可调用对象以作为装饰器，该装饰器返回一个可调用对象来处理对最初的函数或类的调用。

## 编写函数装饰器

### 跟踪调用

In [5]:
class tracter:
    def __init__(self, func):
        self.calls = 0
        self.func = func 
    
    def __call__(self, *args):
        self.calls += 1
        print('call %s to %s' %(self.calls, self.func.__name__))
        self.func(*args)

In [6]:
@tracter
def spam(a, b, c):
    print(a + b + c)

In [7]:
spam(1, 2, 3)

call 1 to spam
6


In [8]:
spam('a', 'b', 'c') # 其实调用的是装饰器的__call__

call 2 to spam
abc


In [9]:
spam.calls

2

其实上面的spam是一个tractor类的实例, 并且这个实例带有自己保存的函数对象和调用计数器，保存了被装饰函数，并且拦截了对被装饰函数的随后调用，以便添加一个统计和打印每次调用的逻辑层。

In [10]:
spam

### 装饰器状态保持方案

#### 类实例属性

In [11]:
class tracter:
    def __init__(self, func):
        self.calls = 0
        self.func = func 
    
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('call %s to %s' %(self.calls, self.func.__name__))
        self.func(*args, **kwargs)

In [12]:
@tracter
def spam(a, b, c):
    print(a + b + c)

In [13]:
@tracter
def eggs(x, y):
    print(x ** y)

In [14]:
spam(1, 2, 3)
spam(a=4, b=5, c=6)

call 1 to spam
6
call 2 to spam
15


In [15]:
eggs(2, 16)
eggs(4, y=4)

call 1 to eggs
65536
call 2 to eggs
256


上述例子中使用了实例属性 `self.calls`显式的保存状态，每个实例都有各自的计数器信息

#### 外层作用域和全局变量

闭包函数常常可以实现相同的效果，特别是用于被装饰器的最初函数这样的静态数据时

In [20]:
calls = 0
def tracer(func):
    def warpper(*args, **kwargs):
        global calls
        calls += 1
        print('call %s to %s'%(calls, func.__name__))
        return func(*args, **kwargs)
    return warpper

In [24]:
@tracer
def spam(a, b, c):
    print(a + b + c)
@tracer
def eggs(x, y):
    print(x ** y)

In [25]:
spam(1, 2, 3)
spam(a = 4, b= 5,c=6)

call 1 to spam
6
call 2 to spam
15


In [26]:
eggs(1, 2) # 因为使用了全局变量

call 3 to eggs
1


#### 外层作用域和非局部变量

In [27]:
def tracer(func):
    calls = 0
    def warpper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print('call %s to %s'%(calls, func.__name__))
        return func(*args, **kwargs)
    return warpper

In [28]:
@tracer
def spam(a, b, c):
    print(a + b + c)
@tracer
def eggs(x, y):
    print(x ** y)

In [29]:
spam(1, 2, 3)
spam(a = 4, b= 5,c=6)

call 1 to spam
6
call 2 to spam
15


In [31]:
eggs(1, 2) # 类似使用了实例属性，每个装饰器有自己的非局部变量

call 2 to eggs
1


#### 函数属性

In [32]:
def tracer(func):
    def warpper(*args, **kwargs):
        warpper.calls += 1
        print('call %s to %s'%(warpper.calls, func.__name__))
        return func(*args, **kwargs)
    warpper.calls = 0
    return warpper

In [33]:
@tracer
def spam(a, b, c):
    print(a + b + c)
@tracer
def eggs(x, y):
    print(x ** y)

In [35]:
spam(1, 2, 3)
print(spam)
spam(a = 4, b= 5,c=6)
print(spam)

call 3 to spam
6
<function tracer.<locals>.warpper at 0x000001A566BB50D8>
call 4 to spam
15
<function tracer.<locals>.warpper at 0x000001A566BB50D8>


In [36]:
eggs(1, 2)

call 1 to eggs
1


#### 添加装饰器参数

In [1]:
import time

In [2]:
def timer(label='', trace=True):
    class Timer:
        def __init__(self, func):
            self.func = func
            self.alltime = 0
        
        def __call__(self, *args, **kargs):
            start = time.clock()
            result = self.func(*args, **kargs)
            elapsed = time.clock() - start 
            self.alltime += elapsed
            if trace:
                format = '%s %s: %.5f, %.5f'
                value = (label, self.func.__name__, elapsed, self.alltime)
                print(format, value)
            return result
        
    return Timer

In [4]:
@timer(label='[CCC]==>')
def listcomp(N):
    return [x*2 for x in range(N)]

listcomp(5000)

%s %s: %.5f, %.5f ('[CCC]==>', 'listcomp', 0.0003523000000313914, 0.0003523000000313914)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98,
 100,
 102,
 104,
 106,
 108,
 110,
 112,
 114,
 116,
 118,
 120,
 122,
 124,
 126,
 128,
 130,
 132,
 134,
 136,
 138,
 140,
 142,
 144,
 146,
 148,
 150,
 152,
 154,
 156,
 158,
 160,
 162,
 164,
 166,
 168,
 170,
 172,
 174,
 176,
 178,
 180,
 182,
 184,
 186,
 188,
 190,
 192,
 194,
 196,
 198,
 200,
 202,
 204,
 206,
 208,
 210,
 212,
 214,
 216,
 218,
 220,
 222,
 224,
 226,
 228,
 230,
 232,
 234,
 236,
 238,
 240,
 242,
 244,
 246,
 248,
 250,
 252,
 254,
 256,
 258,
 260,
 262,
 264,
 266,
 268,
 270,
 272,
 274,
 276,
 278,
 280,
 282,
 284,
 286,
 288,
 290,
 292,
 294,
 296,
 298,
 300,
 302,
 304,
 306,
 308,
 310,
 312,
 314,
 316,
 318,
 320,
 322,
 324,
 326,
 328,
 330,
 332,
 334,
 336,
 338,
 340,
 342,
 344,
 346,
 348,
 350,

## 编写类装饰器

### 单例类

由于装饰器可以拦截实例创建调用，因此它们可以用来管理一个类的所有实例，或者扩充这些实例的接口。

In [5]:
instances = {}

In [6]:
def singleton(aClass):
    def onCall(*args, **kargs):
        if aClass not in instances:
            instances[aClass] = aClass(*args, **kargs) # 装饰器拦截了实例创建调用
        return instances[aClass]
    return onCall

In [7]:
@singleton
class Person:
    def __init__(self, name, hours, rate):
        self.name = name 
        self.hours = hours 
        self.rate = rate 
    
    def pay(self):
        return self.hours * self.rate 

In [8]:
@singleton
class Spam:
    def __init__(self, val):
        self.attr = val

In [10]:
bob = Person('Bob', 40, 10)
print(bob.name, bob.pay())

Bob 400


### 使用装饰器根据接口实现委托

In [13]:
def Tracer(aClass):
    class Wrapper:
        def __init__(self, *args, **kargs):
            self.fetches = 0
            self.wrapped = aClass(*args, **kargs)

        def __getattr__(self, attrname):
            print('Trace: ' + attrname)
            self.fetches += 1
            return getattr(self.wrapped, attrname)
    return Wrapper

In [14]:
@Tracer
class Spam:
    def display(self):
        print('spam!' * 8)

In [15]:
food = Spam()
food.display()

Trace: display
spam!spam!spam!spam!spam!spam!spam!spam!


In [16]:
@Tracer
class Mylist(list): pass

In [19]:
x = Mylist([1, 2, 3, 4])
x.append(5)
print(x.wrapped)

Trace: append
[1, 2, 3, 4, 5]


## 直接管理函数和类

除了用来拦截函数和实例创建调用，装饰器还可以用来管理函数和类对象本身，而不只是管理对他们随后的调用。

In [1]:
registry = {}

In [2]:
def register(obj):
    registry[obj.__name__] = obj 
    return obj 

In [3]:
@register
def spam(x):
    return (x**2)

In [4]:
@register
def ham(x):
    return (x**3)

In [5]:
@register
class Eggs:
    def __init__(self, x):
        self.data = x**4
    def __str__ (self):
        return str(self.data)

In [9]:
for name in registry:
    print(name, '=>', registry[name], type(registry[name]))
    print(registry[name](2))

spam => <function spam at 0x0000015F10A5F798> <class 'function'>
4
ham => <function ham at 0x0000015F110F81F8> <class 'function'>
8
Eggs => <class '__main__.Eggs'> <class 'type'>
16


## 示例："私有"和"公有"属性

### 实现私有属性

实现一个用于类实例属性的private声明，属性存储在一个实例上，或者从一个类继承而来。不接受从被装饰类的外部对这样的属性的获取和修改访问，但是依然允许类自身在其自己的方法中自由地访问那些名称。

In [15]:
traceMe = False
def trace (*args):
    if traceMe:
        print('[' + ' '.join(map(str, args)) + ']')

In [39]:
def Private(*privates):
    def onDecorator(aClass):
        class onInstance:
            def __init__(self, *args, **kargs):
                self.wrapped = aClass(*args, **kargs)
            
            def __getattr__(self, attr):
                trace('get:', attr)
                if attr in privates:
                    raise TypeError('private attrbute fetch: ' + attr)
                else:
                    return getattr(self.wrapped, attr)

            def __setattr__(self, attr, value):
                trace('set:', attr, value)
                if attr == 'wrapped':
                    self.__dict__[attr] = value 
                elif attr in privates:
                    raise TypeError('private attribute change: ' + attr)
                else:
                    setattr(self.wrapped, attr, value)
            
        return onInstance
    return onDecorator

In [40]:
traceMe = True 

In [41]:
@Private('data', 'size')
class Doubler:
    def __init__(self, label, start):
        self.label = label 
        self.data = start 
    
    def size(self):
        return len(self.data)
    
    def double(self):
        for i in range(self.size()):
            self.data[i] = self.data[i] * 2
    
    def display(self):
        print('%s => %s' % (self.label, self.data))

In [42]:
X = Doubler('X is', [1, 2, 3])

[set: wrapped <__main__.Doubler object at 0x0000015F1362E848>]


In [44]:
X.size()

[get: size]


TypeError: private attrbute fetch: size

In [46]:
X.display()

[get: display]
X is => [1, 2, 3]


In [47]:
X.size = lambda S:0

[set: size <function <lambda> at 0x0000015F10994B88>]


TypeError: private attribute change: size

### 公有声明的推广

In [48]:
def accessControl(failIf):
    def onDecorator(aClass):
        class onInstance:
            def __init__(self, *args, **kargs):
                self.__wrapped = aClass(*args, **kargs)
            
            def __getattr__ (self, attr):
                trace('get:', attr)
                if failIf(attr):
                    raise TypeError('private attribute fetch: ' + attr)
                else:
                    return getattr(self.__wrapped, attr)
            
            def __setattr__(self, attr, value):
                trace('set:', attr, value)
                if attr == '_onInstance__wrapped':
                    self.__dict__[attr] = value 
                elif failIf(attr):
                    raise TypeError('private attribute change: ' + attr)
                else:
                    setattr(self.__wrapped, value)
        return onInstance 
    return onDecorator

In [49]:
def Private(*attributes):
    return accessControl(failIf=(lambda attr: attr in attributes))

def Public(*attributes):
    return accessControl(failIf=(lambda attr: attr not in attributes))

In [50]:
@Private('age')
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age

In [51]:
X = Person('Bob', 40)
X.name 

[set: _onInstance__wrapped <__main__.Person object at 0x0000015F12A89388>]
[get: name]


'Bob'

In [52]:
X.age

[get: age]


TypeError: private attribute fetch: age

In [53]:
@Public('name')
class Person:
    def __init__(self, name, age):
        self.name = name 
        self.age = age

In [54]:
X = Person('szq', 18)
X.name 

[set: _onInstance__wrapped <__main__.Person object at 0x0000015F12A88C08>]
[get: name]


'szq'

## 示例：验证函数参数

In [55]:
trace = True

In [56]:
def rangetest(**argchecks):
    def onDecorator(func):
        if not __debug__:
            return func 
        else:
            code = func.__code__
            allargs = code.co_varnames[:code.co_argcount]
            funcname = func.__name__
        
        def onCall(*pargs, **kargs):
            expected = list(allargs)
            positionals = expected[:len(pargs)]

            for (argname, (low, high)) in argchecks.items():
                if argname in kargs:
                    if kargs[argname] < low or kargs[argname] > high:
                        errmsg = '{0} argmust "{1}" not in {2}...{3}'
                        errmsg = errmsg.format(funcname, argname, low, high)
                        raise TypeError(errmsg)
                
                elif argname in positionals:
                    position = positionals.index(argname)
                    if pargs[position] < low or pargs[position] > high:
                        errmsg = '{0} argmust "{1}" not in {2}...{3}'
                        errmsg = errmsg.format(funcname, argname, low, high)
                        raise TypeError(errmsg)
                else:
                    if tarce:
                        print('Argument "{0}" defaulted'.format(argname))
            return func(*pargs, **kargs)
        return onCall
    return onDecorator

In [57]:
@rangetest(age=(0, 120))
def persinfo(name, age):
    print('%s is %s years old' % (name, age))

In [58]:
@rangetest(M=(1, 12), D=(1, 31), Y=(0, 2013))
def birthday(M, D, Y):
    print('birtyday = {0}/{1}/{2}'.format(M, D, Y))

In [59]:
persinfo('szq', 125)

TypeError: persinfo argmust "age" not in 0...120

In [60]:
birthday(5, D=40, Y=1963)

TypeError: birthday argmust "D" not in 1...31

In [64]:
birthday.__code__.co_varnames

('pargs',
 'kargs',
 'expected',
 'positionals',
 'argname',
 'low',
 'high',
 'errmsg',
 'position')

### 函数内省

In [65]:
def func(a, b, e=True, f=None):
    x = 1
    y = 2

In [66]:
code = func.__code__

In [67]:
code.co_nlocals

6

In [68]:
code.co_varnames

('a', 'b', 'e', 'f', 'x', 'y')

In [69]:
code.co_varnames[:code.co_argcount]

('a', 'b', 'e', 'f')

In [70]:
def func(a:(1, 5), b, c:(0.0, 1.0)):
    print(a+b+c)

In [71]:
func.__annotations__

{'a': (1, 5), 'c': (0.0, 1.0)}